# [numpy.newaxis](https://stackoverflow.com/questions/29241056/how-does-numpy-newaxis-work-and-when-to-use-it)
## `numpy.newaxis`的工作方式及适用场合  
简单地说，[newaxis](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#numpy.newaxis)的作用是给现有的数组**增加一个维度**。例如：   
* **1维**数组将变为**2维**数组
* **2维**数组将变为**3维**数组
* **3维**数组将变为**4维**数组
* **4维**数组将变为**5维**数组      
等等。     

我们在这里利用图示的方法解释一维数组通过添加新维度变为二维数组的过程：      
* 一维数组在列方向（**axis=0**）进行广播变为二维数组：       

![列方向添加维度](img/col.png)     
* 一维数组在行方向（**axis=1**）进行广播变为二维数组：    

![行方向添加维度](img/raw.png)     

当然，二维数组沿深度方向（**axis=3**）进行广播可以变为三维数组：    

![深度方向添加维度](img/depth.png)
***
### 适用场合1：   
当你想要将一个一维数组像上面图中所示一样，在行方向或列方向进行广播变为二维数组，你可能需要用到`np.newaxis`。    
示例代码:   

In [1]:
# 创建一维数组
import numpy as np
arr = np.arange(4)
print('arr数组的维度为：{}'.format(arr.shape))
# 我们沿着第一维添加轴使其成为行向量
row_vec = arr[np.newaxis, :]        # 也可写为arr[None, :]
print('row_vec数组的维度为：{}'.format(row_vec.shape))
# 我们沿着第二维添加轴使其成为列向量
col_vec = arr[:, np.newaxis]        # 也可写为arr[:, None]
print('col_vec数组的维度为：{}'.format(col_vec.shape))

arr数组的维度为：(4,)
row_vec数组的维度为：(1, 4)
col_vec数组的维度为：(4, 1)


### 适用场合2：   
当你需要在一些计算过程中采用numpy的[广播机制](https://blog.csdn.net/wzy628810/article/details/103869550)（例如求两个维度不同的数组之和的时候），你可能需要用到`np.newaxis`。      
示例代码:  
例如你想求以下两数组之和

In [2]:
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([5, 4, 3])

如果你将两数组直接相加，Numpy将产生`ValueError`错误:

In [3]:
x1+x2

ValueError: operands could not be broadcast together with shapes (5,) (3,) 

在这种情况下，你可以使用`np.newaxis`对其中一个数组增加维度，这样子numpy便可利用其[广播机制](https://blog.csdn.net/wzy628810/article/details/103869550)使两个数组变为相同维度了。

In [4]:
x1_new = x1[:, np.newaxis]         # 也可写为x1[:, None]
print('现在，x1_new的维度变为了：{}'.format(x1_new.shape))
print('x1_new的具体内容为：\n{}'.format(x1_new))

现在，x1_new的维度变为了：(5, 1)
x1_new的具体内容为：
[[1]
 [2]
 [3]
 [4]
 [5]]


此时，我们再进行数组相加的运算:

In [5]:
print('x1_new + x2 = \n{}'.format(x1_new + x2))

x1_new + x2 = 
[[ 6  5  4]
 [ 7  6  5]
 [ 8  7  6]
 [ 9  8  7]
 [10  9  8]]


***
同样的，你也可以选择为数组`x2`添加轴:

In [6]:
x2_new = x2[:, np.newaxis]         # 也可写为x2[:, None]
print('现在，x2_new的维度变为了：{}'.format(x2_new.shape))
print('x2_new的具体内容为：\n{}'.format(x2_new))

现在，x2_new的维度变为了：(3, 1)
x2_new的具体内容为：
[[5]
 [4]
 [3]]


此时，我们再进行数组相加的运算:

In [7]:
print('x1 + x2_new = \n{}'.format(x1 + x2_new))

x1 + x2_new = 
[[ 6  7  8  9 10]
 [ 5  6  7  8  9]
 [ 4  5  6  7  8]]


**注意：**通过观察我们可以发现两次计算结果相同（其中一个是另一个的转置）。    
***
### 适用场合3：
和场合1类似，但是你可以多次使用`np.newaxis`将数组扩充到更高维度。对于高阶数组(例如：张量)，有时需要这样的操作。    
示例代码: 

In [8]:
arr = np.arange(5*5).reshape(5,5)
print('arr数组的维度为：{}'.format(arr.shape))
# 将2维数组变为5维数组
arr_5D = arr[np.newaxis, ..., np.newaxis, np.newaxis]     # 也可写为arr[None, ..., None, None]
print('arr_5D数组的维度为：{}'.format(arr_5D.shape))

arr数组的维度为：(5, 5)
arr_5D数组的维度为：(1, 5, 5, 1, 1)


***
* `np.newaxis`和`np.reshape`的不同之处           
`newaxis`也被称为假索引，它允许临时将一个轴添加到多维数组中。       
`np.newaxis`使用切片操作符来创建新数组，然而`np.reshape`是根据所需形状对数组进行形状变换（变换前后的维度必须互相匹配）。      
示例代码:  

In [9]:
A = np.ones((3,4,5,6))
B = np.ones((4,6))
print('A + B[:, np.newaxis, :]的维度为：{}'.format((A + B[:, np.newaxis, :]).shape))

A + B[:, np.newaxis, :]的维度为：(3, 4, 5, 6)


上例中，我们在数组`B`的第一个轴和第二个轴之间临时添加一个轴（这里使用`np.newaxis`填充缺失的轴，只有这样子数组才可以进行广播）。   
***
**小贴士：**你也可以使用`None`代替`np.newaxis`，事实上，他们是相同的对象。

In [10]:
print('np.newaxis和None一样吗：{}'.format(np.newaxis is None))

np.newaxis和None一样吗：True
